# Retrieval Augmented Generation
This notebook provides an introduction to Retrieval Augmented Generation.

For this tutorial, we'll use LlamaIndex, which provides some abstractions over underlying APIs used for building LLM applications. For more about RAG and LlamaIndex's definitions, see [here](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html).

In [14]:
import os
from glob import glob

# We'll load documents that we've already downloaded in the Synthetic Dataset for RAG
data_dir = os.path.join(os.path.abspath(""), ".content")
input_files = glob(os.path.join(data_dir, "*.txt"))
print("Files in folder:")
for f in input_files:
    print(os.path.basename(f))

Files in folder:
San_Jose,_California.txt
Oakland,_California.txt
San_Francisco.txt


In [15]:
from dotenv import load_dotenv

load_dotenv()

True

## A simple RAG with LlamaIndex (using defaults)

In [16]:
from llama_index import ServiceContext, SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms import OpenAI

First, build the index from all the documents we have. Using default chunking, and embedding strategies.

In [17]:
service_context = ServiceContext.from_defaults(llm=OpenAI())
documents = SimpleDirectoryReader(input_files=input_files).load_data("*.txt")
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

Loading files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1061.13file/s]


In [18]:
query_engine = index.as_query_engine()

In [19]:
response = query_engine.query("Who were the first settlers in San Francisco?")
print(response)

The first settlers in San Francisco were the Yelamu group of the Ramaytush people.


In [20]:
response = query_engine.query("Does Oakland have a Chinatwon?")
print(response)

Yes, Oakland has a Chinatown.


In [21]:
question = "Where does San Jose get its name from?"
response = query_engine.query(question)
print("Question:")
print(question)
print("Answer:")
print(str(response))
print("Chunks:")
for node in response.source_nodes:
    print("--------------------------")
    print(str(node.text))
    print("--------------------------")

Question:
Where does San Jose get its name from?
Answer:
San Jose gets its name from el Pueblo de San José de Guadalupe, which means "the Town of Saint Joseph of Guadalupe" in Spanish.
Chunks:
--------------------------
== Name ==
San Jose is named after el Pueblo de San José de Guadalupe (Spanish for 'the Town of Saint Joseph of Guadalupe'), the city's predecessor, which was eventually located in the area of what is now the Plaza de César Chávez. In the 19th century, print publications used the spelling "San José" for both the city and its eponymous township. On December 11, 1943, the United States Board on Geographic Names ruled that the city's name should be spelled "San Jose" based on local usage and the formal incorporated name.In the 1960s and 1970s, some residents and officials advocated for returning to the original spelling of "San José", with the acute accent on the "e", to acknowledge the city's Mexican origin and Mexican-American population. On June 2, 1969, the city adopte

## Evaluation of RAGs using Ragas

### Load the Synthetic Dataset for Evaluating the RAGs

In [22]:
from datasets import Dataset

dataset = Dataset.load_from_disk("rag_synth_data.hf")
print(f"Dataset contains {len(dataset)} rows")

Dataset contains 47 rows


In [23]:
dataset

Dataset({
    features: ['query', 'reference_contexts', 'reference_answer', 'reference_answer_by', 'query_by'],
    num_rows: 47
})

## Ragas 
You need the following columns for Ragas Evaluation
- question: list[str] - These are the questions your RAG pipeline will be evaluated on.
- answer: list[str] - The answer generated from the RAG pipeline and given to the user.
- contexts: list[list[str]] - The contexts that were passed into the LLM to answer the question.
- ground_truths: list[list[str]] - The ground truth answer to the questions. (only required if you are using context_recall)


## Ragas Metrics:
The harmonic mean of these 4 aspects gives you the ragas score which is a single measure of the performance of your QA system across all the important aspects.
- faithfulness - the factual consistency of the answer to the context base on the question.
- context_precision - a measure of how relevant the retrieved context is to the question. Conveys quality of the retrieval pipeline.
- answer_relevancy - a measure of how relevant the answer is to the question
- context_recall: measures the ability of the retriever to retrieve all the necessary information needed to answer the question.

In [24]:
from llama_index import ServiceContext
from llama_index.evaluation import CorrectnessEvaluator
from llama_index.llms import OpenAI
from ragas.metrics import (
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness,
)

In [37]:
from statistics import harmonic_mean, mean

import nest_asyncio
from llama_index.embeddings import OpenAIEmbedding
from ragas import evaluate
from tqdm import tqdm

nest_asyncio.apply()


def run_eval(embed_model=None, llm_model=None, dimensions=None):
    questions = []
    answers = []
    contexts = []
    ground_truths = []

    correctness_scores = []

    if embed_model:
        if dimensions:
            embedding_model = OpenAIEmbedding(model=embed_model, dimensions=dimensions)
        else:
            embedding_model = OpenAIEmbedding(model=embed_model)
        if llm_model:
            the_service_context = ServiceContext.from_defaults(embed_model=embedding_model, llm=OpenAI(model=llm_model))
        else:
            the_service_context = ServiceContext.from_defaults(embed_model=embedding_model, llm=OpenAI())
    else:
        if llm_model:
            the_service_context = ServiceContext.from_defaults(llm=OpenAI(model=llm_model))
        else:
            the_service_context = ServiceContext.from_defaults(llm=OpenAI())

    documents = SimpleDirectoryReader(input_files=input_files).load_data("*.txt")
    index = VectorStoreIndex.from_documents(documents, service_context=the_service_context)
    query_engine = index.as_query_engine()

    service_context = ServiceContext.from_defaults(llm=OpenAI())
    evaluator = CorrectnessEvaluator(service_context=service_context)

    for row in tqdm(dataset):
        # The Question
        question = row["query"]
        questions.append(question)

        # The Answer
        response = query_engine.query(question)
        answer = str(response)
        answers.append(answer)

        # Contexts
        context = []
        for node in response.source_nodes:
            context.append(str(node.text))
        contexts.append(context)

        # Ground Truth
        actual_answer = row["reference_answer"]
        ground_truths.append([actual_answer])

        # Correctness with llama-index
        correctness = evaluator.evaluate(
            query=question,
            response=answer,
            reference=actual_answer,
        )
        correctness_scores.append(correctness.score)

    eval_dataset = Dataset.from_dict(
        {"question": questions, "contexts": contexts, "answer": answers, "ground_truths": ground_truths}
    )

    result = evaluate(
        eval_dataset,
        metrics=[
            context_precision,
            faithfulness,
            answer_relevancy,
            context_recall,
        ],
    )
    baseline_ragas = result
    ragas_score = harmonic_mean(list(result.values()))

    return mean(correctness_scores), baseline_ragas, ragas_score

In [28]:
baseline_correctness, baseline_ragas, baseline_score = run_eval(
    embed_model="text-embedding-ada-002", llm_model="gpt-3.5-turbo"
)
print("Baseline Correctness Score:", baseline_correctness)
print("Baseline Ragas Scores:", baseline_ragas)
print("Baseline Overall Ragas Scores:", baseline_score)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [05:00<00:00,  6.39s/it]


evaluating with [context_precision]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:12<00:00,  3.24s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [faithfulness]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:19<00:00, 19.97s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [answer_relevancy]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:17<00:00,  4.42s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_recall]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:56<00:00, 14.03s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Baseline Correctness Score: 3.5
Baseline Ragas Scores: {'context_precision': 0.1702, 'faithfulness': 0.5189, 'answer_relevancy': 0.7469, 'context_recall': 0.8431}
Baseline Overall Ragas Scores: 0.38732844991231846


In [29]:
small_35t_correctness, small_35t_ragas, small_35t_score = run_eval(
    embed_model="text-embedding-3-small", llm_model="gpt-3.5-turbo"
)
print("Text Embedding 3 small + GPT 3.5 Turbo Small Correctness Score:", small_35t_correctness)
print("Text Embedding 3 small + GPT 3.5 Turbo Small Ragas Score:", small_35t_ragas)
print("Text Embedding 3 small + GPT 3.5 Turbo Overall Ragas Score:", small_35t_score)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [04:56<00:00,  6.31s/it]


evaluating with [context_precision]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.57s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [faithfulness]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:10<00:00, 17.64s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:18<00:00,  4.63s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_recall]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:07<00:00, 16.98s/it]

Text Embedding 3 small + GPT 3.5 Turbo Small Correctness Score: 3.627659574468085
Text Embedding 3 small + GPT 3.5 Turbo Small Ragas Score: {'context_precision': 0.2021, 'faithfulness': 0.5059, 'answer_relevancy': 0.7927, 'context_recall': 0.8095}
Text Embedding 3 small + GPT 3.5 Turbo Overall Ragas Score: 0.4245976398525742



/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [30]:
large_35t_correctness, large_35t_ragas, large_35t_score = run_eval(
    embed_model="text-embedding-3-large", llm_model="gpt-3.5-turbo"
)
print("Text Embedding 3 Large Correctness Score:", large_35t_correctness)
print("Text Embedding 3 Large Ragas Score:", large_35t_ragas)
print("Text Embedding 3 Large Overall Ragas Score:", large_35t_score)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [05:04<00:00,  6.48s/it]


evaluating with [context_precision]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:12<00:00,  3.09s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [faithfulness]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:15<00:00, 18.87s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:19<00:00,  4.82s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_recall]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:02<00:00, 15.51s/it]

Text Embedding 3 Large Correctness Score: 3.648936170212766
Text Embedding 3 Large Ragas Score: {'context_precision': 0.1702, 'faithfulness': 0.5038, 'answer_relevancy': 0.8044, 'context_recall': 0.7630}
Text Embedding 3 Large Overall Ragas Score: 0.3841049958861384



/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [31]:
small_4_correctness, small_4_ragas, small_4_score = run_eval(embed_model="text-embedding-3-small", llm_model="gpt-4")
print("Text Embedding 3 + GPT 4 Correctness Score:", small_4_correctness)
print("Text Embedding 3 + GPT 4 Ragas Score:", small_4_ragas)
print("Text Embedding 3 + GPT 4 Overall Ragas Score:", small_4_score)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [08:53<00:00, 11.35s/it]


evaluating with [context_precision]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.61s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [faithfulness]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:53<00:00, 13.35s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [answer_relevancy]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:20<00:00,  5.14s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_recall]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:09<00:00, 17.49s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Text Embedding 3 + GPT 4 Correctness Score: 2.6382978723404253
Text Embedding 3 + GPT 4 Ragas Score: {'context_precision': 0.2021, 'faithfulness': 0.5534, 'answer_relevancy': 0.5839, 'context_recall': 0.8130}
Text Embedding 3 + GPT 4 Overall Ragas Score: 0.412499768638822


In [32]:
small_4t_correctness, small_4t_ragas, small_4t_score = run_eval(
    embed_model="text-embedding-3-small", llm_model="gpt-4-turbo-preview"
)
print("Text Embedding 3 + GPT 4 Turbo Small Correctness Score:", small_4t_correctness)
print("Text Embedding 3 + GPT 4 Turbo Small Ragas Score:", small_4t_ragas)
print("Text Embedding 3 + GPT 4 Turbo Ragas Score:", small_4t_score)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [13:46<00:00, 17.59s/it]


evaluating with [context_precision]


  0%|                                                                                                                                                                   | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.11/site-packages/pydantic/_internal/_core_utils.py:207: RuntimeWarning: coroutine 'CorrectnessEvaluator.aevaluate' was never awaited
  return f(schema, self._walk)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.94s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [faithfulness]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:05<00:00, 16.31s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [answer_relevancy]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:19<00:00,  4.84s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_recall]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:10<00:00, 17.68s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Text Embedding 3 + GPT 4 Turbo Small Correctness Score: 3.723404255319149
Text Embedding 3 + GPT 4 Turbo Small Ragas Score: {'context_precision': 0.2021, 'faithfulness': 0.5595, 'answer_relevancy': 0.7303, 'context_recall': 0.7934}
Text Embedding 3 + GPT 4 Turbo Ragas Score: 0.42714651703774953


In [45]:
large_4t_correctness, large_4t_ragas, large_4t_score = run_eval(
    embed_model="text-embedding-3-large", llm_model="gpt-4-turbo-preview"
)
print("Text Embedding 3 Large + GPT 4 Turbo Small Correctness Score:", large_4t_correctness)
print("Text Embedding 3 Large + GPT 4 Turbo Small Ragas Score:", large_4t_ragas)
print("Text Embedding 3 Large + GPT 4 Turbo Ragas Score:", large_4t_score)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [15:57<00:00, 20.38s/it]


evaluating with [context_precision]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:15<00:00,  3.88s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [faithfulness]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:04<00:00, 16.06s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [answer_relevancy]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:18<00:00,  4.72s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_recall]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:05<00:00, 16.48s/it]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Text Embedding 3 Large + GPT 4 Turbo Small Correctness Score: 3.6382978723404253
Text Embedding 3 Large + GPT 4 Turbo Small Ragas Score: {'context_precision': 0.1702, 'faithfulness': 0.5755, 'answer_relevancy': 0.7250, 'context_recall': 0.7523}
Text Embedding 3 Large + GPT 4 Turbo Ragas Score: 0.3875577131648241


In [41]:
large_256_4t_correctness, large_256_4t_ragas, large_256_4t_score = run_eval(
    embed_model="text-embedding-3-small", dimensions=256, llm_model="gpt-4-turbo-preview"
)
print("Text Embedding 3 Large (256) + GPT 4 Turbo Small Correctness Score:", large_256_4t_correctness)
print("Text Embedding 3 Large (256) + GPT 4 Turbo Small Ragas Score:", large_256_4t_ragas)
print("Text Embedding 3 Large (256) + GPT 4 Turbo Ragas Score:", large_256_4t_score)

Loading files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 730.12file/s]


TypeError: Embeddings.create() got an unexpected keyword argument 'dimensions'

In [33]:
import pandas as pd

In [53]:
comparison = {
    "Embedding Model": [
        "text-embedding-ada-002",
        "text-embedding-3-small",
        "text-embedding-3-large",
        "text-embedding-3-small",
        "text-embedding-3-small",
        "text-embedding-3-large",
    ],
    "LLM Model": [
        "gpt-3.5-turbo",
        "gpt-3.5-turbo",
        "gpt-3.5-turbo",
        "gpt-4",
        "gpt-4-turbo-preview",
        "gpt-4-turbo-preview",
    ],
    "Correctness": [
        baseline_correctness,
        small_35t_correctness,
        large_35t_correctness,
        small_4_correctness,
        small_4t_correctness,
        large_4t_correctness,
    ],
    "Ragas Score": [baseline_score, small_35t_score, large_35t_score, small_4_score, small_4t_score, large_4t_score],
}
df = pd.DataFrame.from_dict(comparison)
df.head(n=10)

,Embedding Model,LLM Model,Correctness,Ragas Score
0,text-embedding-ada-002,gpt-3.5-turbo,3.500000,0.387328
1,text-embedding-3-small,gpt-3.5-turbo,3.627660,0.424598
2,text-embedding-3-large,gpt-3.5-turbo,3.648936,0.384105
3,text-embedding-3-small,gpt-4,2.638298,0.412500
4,text-embedding-3-small,gpt-4-turbo-preview,3.723404,0.427147
5,text-embedding-3-large,gpt-4-turbo-preview,3.638298,0.387558
